In [3]:
import requests
from time import sleep
import pandas as pd
import unicodedata
import string
import urllib
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import gspread
import json

from oauth2client.service_account import ServiceAccountCredentials 
from gspread_dataframe import get_as_dataframe, set_with_dataframe

import ssl
from datetime import datetime as dt
ssl._create_default_https_context = ssl._create_unverified_context
# from utils import *
import datetime
import time
import os

In [36]:
# 並び人数データの取得 
report_df = pd.read_csv(r'csv/report1685968221958.csv')
#提案区分を削除
report_df = report_df.drop('提案区分', axis=1)
#欠損値を削除
report_df = report_df.dropna()
report_df

,イベント日,店舗名／法人名,都道府県,一般並び(スロ),並び人数(スロ)
0,2023/01/01,アスカ杉戸店,埼玉県,49.0,48.0
1,2023/01/01,一番舘草加店,埼玉県,74.0,152.0
2,2023/01/01,アリーナ平塚店,埼玉県,27.0,12.0
3,2023/01/01,ジアスセンター南,神奈川県,23.0,198.0
4,2023/01/01,オータ川島店,埼玉県,25.0,169.0
...,...,...,...,...,...
2251,2023/05/29,スーパーBOSS,栃木県,21.0,35.0
2252,2023/05/29,KING BOSS 1000,栃木県,25.0,33.0
2253,2023/05/30,第1新効,群馬県,8.0,9.0
2254,2023/05/30,イーグル袖ヶ浦店,千葉県,40.0,64.0


In [37]:
##検索する店舗数
len(list(report_df['店舗名／法人名'].unique()))

249

In [11]:
#店舗名の表記ゆれを修正するために一旦グーグルで検索して店舗名を取得する
anaslo_scraping_error_tenpo_name_list = []
browser = webdriver.Chrome(ChromeDriverManager().install())
serched_df = pd.DataFrame(index=[], columns=[])
slopachi_and_anaslo_tenpo_name_dict = {}
error_tenpo_name_dict = {}
#no_scraping_anaslo_tenpo_name = []
for slopachi_tenpo_name in sorted(list(report_df['店舗名／法人名'].unique())):
    url = f"https://www.google.com/search?q=アナスロ  {slopachi_tenpo_name} データ一覧&hl=ja&source=hp/"
    print(url)
    try:
        browser.get(url)
        html = browser.page_source.encode('utf-8')
        
        elems_rec = browser.find_elements(By.CLASS_NAME, 'MjjYud')
        elems_rec[0].text.split('\n')
        print('アナスロと出る部分',elems_rec[0].text.split('\n')[1])

        scraping_anaslo_name = elems_rec[0].text.split('\n')[0]
        slopachi_and_anaslo_tenpo_name_dict[slopachi_tenpo_name] = scraping_anaslo_name

        time.sleep(2)
    except Exception as e:
        print('error',e)
        error_tenpo_name_dict[slopachi_tenpo_name] = e
        continue

C:\Users\tsc95\AppData\Local\Temp\ipykernel_24760\2874368824.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


https://www.google.com/search?q=アナスロ  123横浜西口店 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  ARROW平塚店 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  BIG BOSS 1000 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  BIG PANDORA井土ヶ谷店新館 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  BIG PANDORA浅草店 本館 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  D’STATIONWAKO鹿島店 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  D’STATION伊勢崎店 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  D’STATION坂戸店 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  D’STATION太田矢島店 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  D’STATION富岡店 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  D’STATION沼田総本店 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://

In [ ]:
convert_tenponame_dict_df = pd.DataFrame(slopachi_and_anaslo_tenpo_name_dict.values(), index=slopachi_and_anaslo_tenpo_name_dict.keys())
convert_tenponame_dict_df = convert_tenponame_dict_df.reset_index()
convert_tenponame_dict_df  = convert_tenponame_dict_df.rename(columns={'index': '変換前',0: '変換後'})
convert_tenponame_dict_df = convert_tenponame_dict_df[convert_tenponame_dict_df['変換後'].str.contains('データ一覧')]
convert_tenponame_dict_df['変換後'] = convert_tenponame_dict_df['変換後'].apply(lambda x: x.replace(' データ一覧',''))
convert_tenponame_dict_df.to_csv(r'csv\20230605_並び予測_重回帰分析_アナスロ店舗名変換データ一覧.csv')
convert_tenponame_dict_df
#データ一覧のみを含む店舗名法人名とアナスロ店舗名の辞書型
extraxt_convert_tenponame_dict = dict(zip(convert_tenponame_dict_df['変換前'], convert_tenponame_dict_df['変換後']))
extraxt_convert_tenponame_dict

In [41]:
convert_tenponame_dict_df = pd.read_csv(r'csv\20230605_並び予測_重回帰分析用_アナスロ店舗名変換データ一覧.csv', index_col=0)
convert_tenponame_dict_df

,変換前,変換後
0,123横浜西口店,123横浜西口店
1,ARROW平塚店,ARROW平塚店
2,BIG BOSS 1000,BIGBOSS1000
3,BIG PANDORA井土ヶ谷店新館,BIG PANDORA井土ヶ谷店 新館
4,BIG PANDORA浅草店 本館,BIG PANDORA浅草店 本館
...,...,...
244,第一プラザ西浦和店,第一プラザ西浦和店
245,第一プラザ霞ヶ関店,第一プラザ霞ヶ関店
246,虹のある街北本店,虹のある街北本店
247,鶴見UNO,鶴見UNO


In [42]:
prefecture_name_list = '''北海道,青森県,岩手県,宮城県,秋田県,山形県,福島県,茨城県,\
栃木県,群馬県,埼玉県,千葉県,東京都,神奈川県,新潟県,富山県,石川県,福井県,山梨県,\
長野県,岐阜県,静岡県,愛知県,三重県,滋賀県,京都府,大阪府,兵庫県,奈良県,和歌山県,\
鳥取県,島根県,岡山県,広島県,山口県,徳島県,香川県,愛媛県,高知県,福岡県,佐賀県,\
長崎県,熊本県,大分県,宮崎県,鹿児島県,沖縄県'''.split(',')

In [31]:
import requests
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import datetime
import time
import unicodedata
import string
import urllib
def removal_text(text:str)-> str:
    text = unicodedata.normalize("NFKC", text)
    text = text.translate(str.maketrans( '', '',string.punctuation  + '！'+ '　'+ ' '+'・'+'～' + '‐'))
    return text
def post_line_text(message:str,token:str)-> None:
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload)
    
def post_line_text_and_image(message:str,image_path:str,token:str):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files)
    
def TablePlot(df:pd.DataFrame,outputPath:str,w:int,h:int,fontsize:int,title:str)-> None:
    plt.rcParams["font.size"] = fontsize
    plt.rcParams["font.weight"] = "bold"
    fig, ax = plt.subplots(figsize=(w,h))
    ax.axis('off')
    ax.table(cellText=df.values,
             colLabels=df.columns,
             loc='center',
             bbox=[0,0,1,1])
    ax.set_title(f'{title}',
             fontweight ="bold")
    plt.savefig(outputPath)

##都道府県からアナスロ店舗名とアナスロ店舗URLを取得
tenpo_url_df = pd.DataFrame(index=[], columns=[])
yesterday = datetime.date.today() + datetime.timedelta(days=-1)
browser = webdriver.Chrome(ChromeDriverManager().install())
parlar_name_parlar_url_dict = {}
for prefecture_name in prefecture_name_list:
    url = f'https://ana-slo.com/%E3%83%9B%E3%83%BC%E3%83%AB%E3%83%87%E3%83%BC%E3%82%BF/{prefecture_name}/'
    browser.get(url)
    html = browser.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'lxml')
    for element in soup.find(class_="dataTable no-footer").find_all("a"):
        #print(element.string,element['href'])
        parlar_name = element.string
        parlar_name_url = element['href']
        parlar_name_parlar_url_dict[parlar_name] = parlar_name_url
        tenpo_url_df = pd.concat([tenpo_url_df,pd.DataFrame([[prefecture_name,parlar_name,parlar_name_url]])],ignore_index=True)
    #break
tenpo_url_df.columns = ['prefecture','anaslo_tenpo_name','url']

C:\Users\tsc95\AppData\Local\Temp\ipykernel_24760\1681855874.py:50: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [35]:
tenpo_url_df['url_tenpo_name'] = tenpo_url_df['url'].map(lambda x: urllib.parse.unquote((x.split('/')[-2])).replace('-データ一覧',''))
tenpo_url_df.to_csv(r'csv\prefecture_convert_anaslo_url.csv',index=False)

In [45]:
tenpo_url_df = pd.read_csv(r'csv\prefecture_convert_anaslo_url.csv')
tenpo_url_df.head()

,prefecture,anaslo_tenpo_name,url,url_tenpo_name
0,北海道,BIG大王東店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,big大王東店
1,北海道,CREV伊達店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,crev伊達店
2,北海道,EBH,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,ebh
3,北海道,INDY函館,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,indy函館
4,北海道,KEIO湯川店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,keio湯川店


In [47]:
##アナスロ店舗名に変換した店舗名を追加
report_df['アナスロ店舗名'] = report_df['店舗名／法人名'].map(extraxt_convert_tenponame_dict)
#欠損値を削除
report_df.dropna(subset=['アナスロ店舗名'],inplace=True)
report_df

,イベント日,店舗名／法人名,都道府県,一般並び(スロ),並び人数(スロ),アナスロ店舗名
0,2023/01/01,アスカ杉戸店,埼玉県,49.0,48.0,アスカ杉戸店
1,2023/01/01,一番舘草加店,埼玉県,74.0,152.0,一番舘草加店
2,2023/01/01,アリーナ平塚店,埼玉県,27.0,12.0,アリーナ平塚店
3,2023/01/01,ジアスセンター南,神奈川県,23.0,198.0,ジアスセンター南
4,2023/01/01,オータ川島店,埼玉県,25.0,169.0,オータ川島店
...,...,...,...,...,...,...
2250,2023/05/29,ニュークラウン川越2号店,埼玉県,9.0,96.0,ニュークラウン川越2号店
2251,2023/05/29,スーパーBOSS,栃木県,21.0,35.0,SuperBOSS
2252,2023/05/29,KING BOSS 1000,栃木県,25.0,33.0,KING BOSS 1000
2254,2023/05/30,イーグル袖ヶ浦店,千葉県,40.0,64.0,イーグルNO.1


In [49]:
len(report_df['アナスロ店舗名'].unique())


215

In [ ]:
#イベント日をdatetime型に変換
report_df['イベント日'] = report_df['イベント日'].map(lambda x: datetime.datetime.strptime(x, '%Y/%m/%d'))

In [ ]:
anaslo_scraping_error_tenpo_name_list = []
browser = webdriver.Chrome(ChromeDriverManager().install())
serched_df = pd.DataFrame(index=[], columns=[])
# for target_day in list(sammy_sf_report_df['イベント日'].unique()):
#     print(target_day)
error_tenpo_name_list = []
anaslo_scraping_error_tenpo_name_list  = []
for i, row in sammy_sf_report_df.iterrows():
    try:
        tenpo_name = row['アナスロ店舗名']
        date = row['イベント日'].to_pydatetime().strftime('%Y-%m-%d')
        anaslo_parlar_name_url = parlar_name_parlar_url_dict[tenpo_name]
        anaslo_url_parlarname =urllib.parse.unquote((anaslo_parlar_name_url.split('/')[-2])).replace('-データ一覧','')
        url = f"https://ana-slo.com/{date}-{anaslo_url_parlarname}-data/"
        print(url)
        #print('データ取得中...30秒ほど時間がかかります。')
        browser.get(url)
        html = browser.page_source.encode('utf-8')
        dfs = pd.read_html(html)
        #display(tenpo_df)
        #time.sleep(1)
        elements = browser.find_element(By.CLASS_NAME, 'st-catgroup')
        prefecture = elements.text.split(' ')[-1]
        for df in  dfs:
        #print(df.columns)
            if '機種名' in list(df.columns):
                ichiran_df = df
                ichiran_df['店舗名'] = tenpo_name
                #ichiran_df['都道府県'] = prefecture
                ichiran_df['機種名'] = ichiran_df['機種名'].map(removal_text)
                first_column = ichiran_df.pop('店舗名')
                ichiran_df.insert(0,'店舗名',first_column)
                break
        tenpo_day_data_df = ichiran_df
        tenpo_day_data_df = tenpo_day_data_df.sort_values('台番号')
        tenpo_day_data_df['日付'] = f"{date}"
        serched_df = pd.concat([serched_df,tenpo_day_data_df],axis=0)
    except Exception as e:
        anaslo_scraping_error_tenpo_name_list.append(tenpo_name)
        print('エラー店舗',tenpo_name,e)
        continue
    # if i > 2:
    #     break

In [98]:
urls = []
for i,row in report_df.iterrows():
    serch_date = row['イベント日'].to_pydatetime().strftime('%Y-%m-%d')
    anaslo_tenpo_name = row['アナスロ店舗名']
    anaslo_parlar_name_url = parlar_name_parlar_url_dict[anaslo_tenpo_name]
    anaslo_url_parlarname =urllib.parse.unquote((anaslo_parlar_name_url.split('/')[-2])).replace('-データ一覧','')
    
    search_url = url = f"https://ana-slo.com/{serch_date}-{anaslo_url_parlarname}-data/"
    urls.append(search_url)
    #break

In [100]:
concat_df_list = []
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor(15) as executor:
    results = list(executor.map(requests.get, urls[800:]))
print(results)


[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [404]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [404]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [404]>, <Response [200]>, <Response [404]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [404]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response

In [101]:
concat_df_list = []
for search_response in results:
    try:
        soup = BeautifulSoup(search_response.text, "lxml")
        elem = soup.select('#all_data_block')
        dfs = pd.read_html(str(elem))
        for tmp_df in dfs:
            if '機種名' in list(tmp_df.columns):
                break
        tmp_df['anaslo_url_tenpo_name'] = soup.title.text.split(' ')[1]
        tmp_df['日付'] = soup.title.text.split(' ')[0]
        concat_df_list.append(tmp_df)
    except:
        continue
    #break

concat_df = pre_concat_df = pd.concat(concat_df_list,axis=0)
# for column_name in ['合成確率','BB確率','RB確率','台番号','ART確率','BB','RB','ART']:
#     try:
#         concat_df = concat_df.drop([column_name],axis=1)
#     except:
#         pass
# concat_df = concat_df.groupby(['日付','機種名']).mean().sort_values('差枚',ascending=False)#機種毎に集計
# for column_name in ['差枚','G数','BB','RB','ART']:
#     try:
#         concat_df[column_name] = concat_df[column_name].astype(int)
#     except:
#         pass

concat_df

,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,anaslo_url_tenpo_name,日付
0,アイムジャグラーEX-TP,157,741.0,-235.0,2,2,0.0,1/185.2,1/370.5,1/370.5,1/0.0,王子龍龍,2023/03/17
1,アイムジャグラーEX-TP,158,3217.0,1255.0,19,5,0.0,1/134.0,1/169.3,1/643.4,1/0.0,王子龍龍,2023/03/17
2,アイムジャグラーEX-TP,159,2106.0,-152.0,8,3,0.0,1/191.5,1/263.2,1/702.0,1/0.0,王子龍龍,2023/03/17
3,アイムジャグラーEX-TP,160,408.0,-152.0,1,1,0.0,1/204.0,1/408.0,1/408.0,1/0.0,王子龍龍,2023/03/17
4,アイムジャグラーEX-TP,161,7913.0,1692.0,32,33,0.0,1/121.7,1/247.3,1/239.8,1/0.0,王子龍龍,2023/03/17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,バンバンクロス,603,0.0,0.0,0,0,NaN,1/0.0,1/0.0,1/0.0,NaN,出玉王田無店,2023/05/31
296,ピンクパンサーSP,604,66.0,-85.0,0,0,NaN,1/0.0,1/0.0,1/0.0,NaN,出玉王田無店,2023/05/31
297,マッピー,607,2908.0,303.0,13,10,NaN,1/126.4,1/223.7,1/290.8,NaN,出玉王田無店,2023/05/31
298,バキ 強くなりたくば喰らえ!!!,663,1097.0,-982.0,7,5,NaN,1/91.4,1/156.7,1/219.4,NaN,出玉王田無店,2023/05/31


In [102]:
concat_df.to_csv('csv/scraping_data_2.csv',index=False)

In [ ]:
groupby_samai_game_mean_df = concat_df.drop(['機種名'],axis=1)
groupby_samai_game_mean_df = groupby_samai_game_mean_df.groupby('日付').mean()
groupby_samai_game_mean_df.reset_index(inplace=True)
groupby_samai_game_mean_df['差枚'] = groupby_samai_game_mean_df['差枚'].astype(int)
groupby_samai_game_mean_df['G数'] = groupby_samai_game_mean_df['G数'].astype(int)
groupby_samai_game_mean_df['差枚'] = groupby_samai_game_mean_df['差枚'].astype(str)
groupby_samai_game_mean_df['G数'] = groupby_samai_game_mean_df['G数'].astype(str)
samai_list:str = str(groupby_samai_game_mean_df['差枚'].tolist())
gamesuu_list:str = str(groupby_samai_game_mean_df['G数'].tolist())
concat_df = concat_df.rename(columns={'差枚': '平均差枚', 'G数': '平均G数'})

record = [groupby_samai_game_mean_df['差枚'].tolist(),groupby_samai_game_mean_df['G数'].tolist()]
print(record)
ave_tenpo_df = pd.DataFrame(record, columns=target_day_list,index=['平均差枚','平均G数'])
ave_tenpo_df[0:1]  =  ave_tenpo_df[0:1]  + '枚'
ave_tenpo_df[1:2]  =  ave_tenpo_df[1:2]  + 'G'